<a href="https://colab.research.google.com/github/BrunoGomesCoelho/mosquito-networking/blob/master/notebooks/1.3-BrunoGomesCoelho_Colab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Colab console code
"""
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}setInterval(ClickConnect,60000)
"""

In [0]:
import time
start_time = time.time()

In [1]:
COLAB_IDX = 1
TEMPERATURE = "t02"

In [0]:
TESTING = False
COLAB = True

if COLAB:
    BASE_DIR = "/content/drive/My Drive/IC/mosquito-networking/"
else:
    BASE_DIR = "../"

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
sys.path.append("/content/drive/My Drive/IC/mosquito-networking/")

In [0]:
!python3 -m pip install -qr "/content/drive/My Drive/IC/mosquito-networking/drive_requirements.txt"

     |████████████████████████████████| 2.1MB 6.5MB/s 
     |████████████████████████████████| 71kB 24.0MB/s 
     |████████████████████████████████| 122kB 49.5MB/s 
     |████████████████████████████████| 5.7MB 41.1MB/s 
     |████████████████████████████████| 51kB 20.8MB/s 
     |████████████████████████████████| 61kB 24.0MB/s 
     |████████████████████████████████| 51kB 21.1MB/s 




- - - 

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

# Trying out a full pytorch experiment, with tensorboard, // processing, etc

In [0]:
# OPTIONAL: Load the "autoreload" extension so that code can change
#%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
#%autoreload 2

import numpy as np
import pandas as pd
from src.data import make_dataset
from src.data import read_dataset
from src.data import util
from src.data.colab_dataset import MosquitoDatasetColab
import joblib

from torchsummary import summary

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Experiment params

In [0]:
# Parameters
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 0}
max_epochs = 1

if TESTING:
    params["num_workers"] = 0

version = !python3 --version
version = version[0].split(".")[1]

if int(version) < 7 and params["num_workers"]:
    print("WARNING\n"*10)
    print("Parallel execution only works for python3.7 or above!")
    print("Running in parallel with other versions is not guaranted to work")
    print("See https://discuss.pytorch.org/t/valueerror-signal-number-32-out-of-range-when-loading-data-with-num-worker-0/39615/2")

In [0]:
## Load gpu or cpu
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device {device}")

Using device cuda:0


# load data

In [0]:
# Load scaler
#scaler = joblib.load("../data/interim/scaler.pkl")
scaler = joblib.load(BASE_DIR + f"data/interim/scaler_{TEMPERATURE}.pkl")

In [0]:
data = np.load(BASE_DIR + f"data/interim/all_wavs_{TEMPERATURE}.npz",
               allow_pickle=True)

In [0]:
# Parse and divide data
train_data = data["train"]
val_data = data["val"]
test_data = data["test"]

x_train = train_data[:, :-1]
y_train = train_data[:, -1]

x_val = val_data[:, :-1]
y_val = val_data[:, -1]

x_test = test_data[:, :-1]
y_test = test_data[:, -1]

In [0]:
# Generators
training_set = MosquitoDataTemperature(x_train, y_train,
                                    device=device, scaler=scaler)
training_generator = torch.utils.data.DataLoader(training_set, **params, 
                                                 pin_memory=True)

val_set = MosquitoDataTemperature(x_val, y_val,
                                device=device, scaler=scaler)

val_generator = torch.utils.data.DataLoader(val_set, **params, 
                                             pin_memory=True)

#sc Generate some example data
temp_generator = torch.utils.data.DataLoader(training_set, **params)

for (local_batch, local_labels) in temp_generator:
    example_x = local_batch
    example_y = local_labels
    break


# Load model

In [0]:
from src.models.BasicMosquitoNet2 import BasicMosquitoNet

In [0]:
# create your optimizer
net = BasicMosquitoNet()
#net.load_state_dict(torch.load(BASE_DIR + 
#f"runs/colab/{COLAB_IDX-1}/model_epoch_90.pt"))
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)

if device.type == "cuda":
    net.cuda()

In [0]:
summary(net, input_size=example_x.shape[1:])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1             [-1, 64, 7309]           8,256
       BatchNorm1d-2             [-1, 64, 7309]             128
            Conv1d-3             [-1, 32, 3527]         262,176
       BatchNorm1d-4             [-1, 32, 3527]              64
            Linear-5                    [-1, 1]          56,417
Total params: 327,041
Trainable params: 327,041
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.08
Forward/backward pass size (MB): 8.86
Params size (MB): 1.25
Estimated Total Size (MB): 10.19
----------------------------------------------------------------


# Start tensorboard

In [0]:
from torch.utils.tensorboard import SummaryWriter
save_path =  BASE_DIR + f"runs/colab/{TEMPERATURE}/{COLAB_IDX}/"
    
# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter(save_path, max_queue=3)

# train function

In [0]:
# Simple train function

def train(net, optimizer, max_epochs, testing=False, testing_idx=0, 
          save_idx=1, save_path=""):
    # Loop over epochs
    last_test_loss = 0
    for epoch in range(max_epochs):
        # Training
        cumulative_train_loss = 0
        cumulative_train_acc = 0
        amount_train_samples = 0

        for idx, (local_batch, local_labels) in enumerate(training_generator):
            amount_train_samples += len(local_batch)
            local_batch, local_labels = util.convert_cuda(local_batch, 
                                                         local_labels,
                                                         device)

            optimizer.zero_grad()   # zero the gradient buffers
            output = net(local_batch)
            loss = criterion(output, local_labels)
            cumulative_train_loss += loss.data.item()
            loss.backward()
            optimizer.step()    # Does the update

            # Stores loss
            pred = output >= 0.5
            cumulative_train_acc += pred.float().eq(local_labels).sum().data.item()
            
            if testing and idx == testing_idx:
                    break
        
        cumulative_train_loss /= (idx+1)
        cumulative_train_acc /= amount_train_samples
        writer.add_scalar("Train Loss", cumulative_train_loss, epoch)
        writer.add_scalar("Train Acc", cumulative_train_acc, epoch)

        # Validation
        with torch.set_grad_enabled(False):
            cumulative_test_loss = 0
            cumulative_test_acc = 0
            amount_test_samples = 0
            
            for idx, (local_batch, local_labels) in enumerate(val_generator):
                amount_test_samples += len(local_batch)
                local_batch, local_labels = util.convert_cuda(local_batch, 
                                                 local_labels,
                                                 device)

                output = net(local_batch)
                loss = criterion(output, local_labels)
                cumulative_test_loss += loss.data.item()
                
                # Stores loss
                pred = output >= 0.5
                cumulative_test_acc += pred.float().eq(local_labels).sum().data.item()

                if testing:
                    break
                
            cumulative_test_loss /= (idx+1)
            cumulative_test_acc /= amount_test_samples
            writer.add_scalar("Val Loss", cumulative_test_loss, epoch)
            writer.add_scalar("Val Acc", cumulative_test_acc, epoch)
        torch.save(net.state_dict(), save_path + f"model_epoch_{epoch}.pt")
            
    writer.close()
    return cumulative_test_loss

In [0]:
%%time
train(net, optimizer, 150, testing=TESTING, save_path=save_path)

In [0]:
print(start_time)
print(time.time() - start_time)